# 1. Data Preprocessing

We will do some data cleaning in this part. The data is present in 'flight_weather_data.csv', which is the combined data of EWR, JFK, and LGA airports of New York, along with the corresponding weather data for those airports. 

Firstly, we will convert date into a numerical day value that ranges from 1 to 365. We will then drop some redundant columns.

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from SMOTE import smote

cur_dir = os.getcwd()


df = pd.read_csv(cur_dir + '/../data/flight_weather_data.csv', low_memory=False)

In [2]:
df

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,...,date,Maximum,Minimum,Average,Departure,HDD,CDD,Precipitation,New Snow,Snow Depth
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,...,2013-01-01,41.0,27.0,34.0,-0.3,31.0,0.0,T,T,0
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,...,2013-01-01,41.0,27.0,34.0,-2.1,31.0,0.0,T,0.0,0
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,...,2013-01-01,41.0,27.0,34.0,-0.6,31.0,0.0,0.00,0.0,0
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,...,2013-01-01,41.0,27.0,34.0,-0.6,31.0,0.0,0.00,0.0,0
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,...,2013-01-01,41.0,27.0,34.0,-2.1,31.0,0.0,T,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336771,2013,9,30,NaN,1455,NaN,NaN,1634,NaN,9E,...,2013-09-30,72.0,52.0,62.0,-1.3,3.0,0.0,0.00,0.0,0
336772,2013,9,30,NaN,2200,NaN,NaN,2312,NaN,9E,...,2013-09-30,73.0,58.0,65.5,-0.2,0.0,1.0,0.00,0.0,0
336773,2013,9,30,NaN,1210,NaN,NaN,1330,NaN,MQ,...,2013-09-30,73.0,58.0,65.5,-0.2,0.0,1.0,0.00,0.0,0
336774,2013,9,30,NaN,1159,NaN,NaN,1344,NaN,MQ,...,2013-09-30,73.0,58.0,65.5,-0.2,0.0,1.0,0.00,0.0,0


In [3]:
def convert_date_to_days_in_365(df):
  # starting day of each month - 1
  month_start_day = {
    1: 0, 2: 31, 3: 59, 4: 90, 5: 120, 6: 151, 
    7: 181, 8: 212, 9: 243, 10: 273, 11: 304, 12: 334
  }

  days_in_365 = [month_start_day[i] for i in df['month']] + df['day']

  return days_in_365

# Convert date to days in range 1 to 365
df['days_in_365'] = convert_date_to_days_in_365(df)

In [4]:
def convert_time_to_minute_of_the_day(df):
  hour = [int(str(i)[:-2]) for i in df['sched_dep_time']]
  minute = [int(str(i)[-2:]) for i in df['sched_dep_time']]

  sched_time_in_min = [i*60 + j for i, j in zip(hour, minute)]

  return sched_time_in_min

# Drop the 'dep_time' column since it is duplicate (We have scheduled time and delay time)
if 'dep_time' in df.columns:
  df.drop('dep_time', axis=1, inplace=True)

df['sched_time_in_min'] = convert_time_to_minute_of_the_day(df)

# Remove rows with missing values in 'dep_delay' column. These are the flights that got cancelled
# TODO: make this work. This is not working
df.dropna(subset=['dep_delay'], inplace=True)

# Apply the 'delay_class' classification to 'dep_delay' column
# This is our target variable for classification
df['delay_class']=['yes' if x > 10 else 'no' for x in df['dep_delay']]


In [5]:
df

,year,month,day,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,...,Average,Departure,HDD,CDD,Precipitation,New Snow,Snow Depth,days_in_365,sched_time_in_min,delay_class
0,2013,1,1,515,2.0,830.0,819,11.0,UA,1545,...,34.0,-0.3,31.0,0.0,T,T,0,1,315,no
1,2013,1,1,529,4.0,850.0,830,20.0,UA,1714,...,34.0,-2.1,31.0,0.0,T,0.0,0,1,329,no
2,2013,1,1,540,2.0,923.0,850,33.0,AA,1141,...,34.0,-0.6,31.0,0.0,0.00,0.0,0,1,340,no
3,2013,1,1,545,-1.0,1004.0,1022,-18.0,B6,725,...,34.0,-0.6,31.0,0.0,0.00,0.0,0,1,345,no
4,2013,1,1,600,-6.0,812.0,837,-25.0,DL,461,...,34.0,-2.1,31.0,0.0,T,0.0,0,1,360,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336765,2013,9,30,2245,-5.0,2334.0,2351,-17.0,B6,1816,...,62.0,-1.3,3.0,0.0,0.00,0.0,0,273,1365,no
336766,2013,9,30,2250,-10.0,2347.0,7,-20.0,B6,2002,...,62.0,-1.3,3.0,0.0,0.00,0.0,0,273,1370,no
336767,2013,9,30,2246,-5.0,2345.0,1,-16.0,B6,486,...,62.0,-1.3,3.0,0.0,0.00,0.0,0,273,1366,no
336768,2013,9,30,2255,12.0,2359.0,2358,1.0,B6,718,...,62.0,-1.3,3.0,0.0,0.00,0.0,0,273,1375,yes


In [6]:
print(str(df[df['delay_class'] == 'no'].delay_class.count()) + " flights were not delayed")
print(str(df[df['delay_class'] == 'yes'].delay_class.count()) + " flights were delayed")

245687 flights were not delayed
82834 flights were delayed


Remove the 'T' (trace) values from 'Precipitation', 'New Snow', and 'Snow Depth' columns  
In meteorological terms, trace means that the rain/snow was too negligible to be recorded  

We add new binary columns for these to preserve information, since negligible rain could mean   
that the weather was cloudy, and that might affect the flight delay probability

In [7]:
def remove_trace(df):
    cols = ['Precipitation', 'New Snow', 'Snow Depth']

    for col in cols:
        df[col + ' Binary'] = ['yes' if i == 'T' or float(i) != 0 else 'no' for i in df[col]]
        df[col] = [0 if i == 'T' else i for i in df[col]]

    return df

df = remove_trace(df)

Finally, we can remove unneeded columns. These are columns which are either useless (such as 'year' where the value is always 2013),  only contain data that is already conveyed in other columns, or are simply too specific and may lead to overfitting (such as individual plane's tail numbers)

In [8]:
df.drop('year', axis=1, inplace=True)
df.drop('month', axis=1, inplace=True)
df.drop('day', axis=1, inplace=True)
df.drop('flight', axis=1, inplace=True)
df.drop('tailnum', axis=1, inplace=True)
df.drop('sched_dep_time', axis=1, inplace=True)
df.drop('dep_delay', axis=1, inplace=True)
df.drop('sched_arr_time', axis=1, inplace=True)
df.drop('arr_delay', axis=1, inplace=True)
df.drop('arr_time', axis=1, inplace=True)
df.drop('hour', axis=1, inplace=True)
df.drop('minute', axis=1, inplace=True)
df.drop('time_hour', axis=1, inplace=True)
df.drop('date', axis=1, inplace=True)
df.drop('HDD', axis=1, inplace=True)
df.drop('CDD', axis=1, inplace=True)

In [9]:
df.to_csv("..\data\cleaned_data.csv", index=False)
print(str(df[df['delay_class'] == 'no'].delay_class.count()) + " flights were not delayed")
print(str(df[df['delay_class'] == 'yes'].delay_class.count()) + " flights were delayed")
df

245687 flights were not delayed
82834 flights were delayed


,carrier,origin,dest,air_time,distance,Maximum,Minimum,Average,Departure,Precipitation,New Snow,Snow Depth,days_in_365,sched_time_in_min,delay_class,Precipitation Binary,New Snow Binary,Snow Depth Binary
0,UA,EWR,IAH,227.0,1400,41.0,27.0,34.0,-0.3,0,0,0,1,315,no,yes,yes,no
1,UA,LGA,IAH,227.0,1416,41.0,27.0,34.0,-2.1,0,0.0,0,1,329,no,yes,no,no
2,AA,JFK,MIA,160.0,1089,41.0,27.0,34.0,-0.6,0.00,0.0,0,1,340,no,no,no,no
3,B6,JFK,BQN,183.0,1576,41.0,27.0,34.0,-0.6,0.00,0.0,0,1,345,no,no,no,no
4,DL,LGA,ATL,116.0,762,41.0,27.0,34.0,-2.1,0,0.0,0,1,360,no,yes,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336765,B6,JFK,SYR,41.0,209,72.0,52.0,62.0,-1.3,0.00,0.0,0,273,1365,no,no,no,no
336766,B6,JFK,BUF,52.0,301,72.0,52.0,62.0,-1.3,0.00,0.0,0,273,1370,no,no,no,no
336767,B6,JFK,ROC,47.0,264,72.0,52.0,62.0,-1.3,0.00,0.0,0,273,1366,no,no,no,no
336768,B6,JFK,BOS,33.0,187,72.0,52.0,62.0,-1.3,0.00,0.0,0,273,1375,yes,no,no,no


Now that we have cleaned data, we can normalize it and divide it into training, testing, and validation sets.

In [10]:
cols = [
 'air_time',
 'distance',
 'Maximum',
 'Minimum',
 'Average',
 'Departure',
 'Precipitation',
 'New Snow',
 'Snow Depth',
 'days_in_365',
 'sched_time_in_min']

scaler = MinMaxScaler()
df[cols] = scaler.fit_transform(df[cols])

df

,carrier,origin,dest,air_time,distance,Maximum,Minimum,Average,Departure,Precipitation,New Snow,Snow Depth,days_in_365,sched_time_in_min,delay_class,Precipitation Binary,New Snow Binary,Snow Depth Binary
0,UA,EWR,IAH,0.306667,0.269223,0.259259,0.213333,0.233766,0.392157,0.0,0.0,0.0,0.000000,0.013169,no,yes,yes,no
1,UA,LGA,IAH,0.306667,0.272486,0.259259,0.213333,0.233766,0.352941,0.0,0.0,0.0,0.000000,0.025461,no,yes,no,no
2,AA,JFK,MIA,0.207407,0.205792,0.259259,0.213333,0.233766,0.385621,0.0,0.0,0.0,0.000000,0.035119,no,no,no,no
3,B6,JFK,BQN,0.241481,0.305119,0.259259,0.213333,0.233766,0.385621,0.0,0.0,0.0,0.000000,0.039508,no,no,no,no
4,DL,LGA,ATL,0.142222,0.139099,0.259259,0.213333,0.233766,0.352941,0.0,0.0,0.0,0.000000,0.052678,no,yes,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336765,B6,JFK,SYR,0.031111,0.026310,0.641975,0.546667,0.597403,0.370370,0.0,0.0,0.0,0.747253,0.935031,no,no,no,no
336766,B6,JFK,BUF,0.047407,0.045074,0.641975,0.546667,0.597403,0.370370,0.0,0.0,0.0,0.747253,0.939421,no,no,no,no
336767,B6,JFK,ROC,0.040000,0.037528,0.641975,0.546667,0.597403,0.370370,0.0,0.0,0.0,0.747253,0.935909,no,no,no,no
336768,B6,JFK,BOS,0.019259,0.021823,0.641975,0.546667,0.597403,0.370370,0.0,0.0,0.0,0.747253,0.943810,yes,no,no,no


We split our data into a training set (60%), and test and validation sets (20% each)

In [11]:
from sklearn.model_selection import train_test_split

X = df.loc[:, df.columns != 'delay_class']
y = df['delay_class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

train_set = X_train.join(y_train)
val_set = X_val.join(y_val)
test_set = X_test.join(y_test)

train_set.to_csv(r"..\data\train_set_natural.csv", index=False)
val_set.to_csv(r"..\data\validation_set.csv", index=False)
test_set.to_csv(r"..\data\test_set.csv", index=False)

train_set

,carrier,origin,dest,air_time,distance,Maximum,Minimum,Average,Departure,Precipitation,New Snow,Snow Depth,days_in_365,sched_time_in_min,Precipitation Binary,New Snow Binary,Snow Depth Binary,delay_class
91742,VX,EWR,LAX,0.521481,0.484193,0.185185,0.213333,0.194805,0.213508,0.067332,0.348485,0.0,0.942308,0.215101,yes,yes,no,no
154582,AA,JFK,LAS,0.451852,0.442178,0.333333,0.280000,0.305195,0.342048,0.000000,0.000000,0.0,0.214286,0.632133,no,no,no,no
52095,US,EWR,CLT,0.090370,0.091577,0.543210,0.333333,0.441558,0.331155,0.000000,0.000000,0.0,0.824176,0.079017,no,no,no,no
215618,AA,JFK,SAN,0.459259,0.482562,0.604938,0.466667,0.538961,0.283224,0.049875,0.000000,0.0,0.392857,0.654083,yes,no,no,yes
147140,UA,LGA,IAH,0.284444,0.272486,0.444444,0.426667,0.435065,0.568627,0.197007,0.000000,0.0,0.192308,0.640913,yes,no,no,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165932,MQ,JFK,CMH,0.091852,0.082195,0.530864,0.346667,0.441558,0.490196,0.000000,0.000000,0.0,0.247253,0.772608,yes,no,no,yes
72552,DL,JFK,SJU,0.247407,0.309606,0.395062,0.333333,0.363636,0.363834,0.000000,0.000000,0.0,0.884615,0.100966,no,no,no,no
232354,B6,JFK,SJU,0.284444,0.309606,0.753086,0.720000,0.740260,0.490196,0.004988,0.000000,0.0,0.442308,0.913082,yes,no,no,no
65182,EV,EWR,MSP,0.229630,0.189272,0.407407,0.360000,0.383117,0.335512,0.000000,0.000000,0.0,0.862637,0.197542,no,no,no,no


Now we can expand using SMOTE-NC

In [12]:
#TODO: Use SMOTE

In [13]:
min_class = train_set.loc[train_set['delay_class']=='yes']
maj_class = train_set.loc[train_set['delay_class']=='no']

art_data = smote(min_class, 5, 5)

art_data.to_csv(r"..\data\train_set_artificial.csv", index=False)

Randomized!


,carrier,origin,dest,air_time,distance,Maximum,Minimum,Average,Departure,Precipitation,New Snow,Snow Depth,days_in_365,sched_time_in_min,Precipitation Binary,New Snow Binary,Snow Depth Binary,delay_class
0,AA,JFK,LAS,0.435556,0.462370,0.561728,0.480000,0.522727,0.285403,0.028678,0.0,0.0,0.380495,0.643108,yes,no,no,yes
1,UA,LGA,DEN,0.286667,0.272486,0.469136,0.346667,0.409091,0.607843,0.208229,0.0,0.0,0.137363,0.638718,yes,no,no,yes
2,EV,EWR,RIC,0.045185,0.040689,0.185185,0.053333,0.116883,0.200436,0.000000,0.0,0.7,0.109890,0.765145,no,no,yes,yes
3,EV,EWR,DCA,0.023704,0.023863,0.339506,0.260000,0.298701,0.300654,0.000000,0.0,0.0,0.212912,0.210272,no,no,no,yes
4,B6,JFK,SFO,0.420000,0.436875,0.425926,0.420000,0.422078,0.348584,0.000000,0.0,0.0,0.840659,0.789728,no,no,no,yes
